# Feature Scaling

One of the most common transformations to make on continuous data is to scale each feature so that they all share similar ranges. For instance, we can scale each feature so each one has a mean of 0 and a standard deviation of 1. We can also scale each feature so that the minimum is 0 and maximum is 1. These feature scaling strategies are called **standardization** and **min-max** respectively, and along with a few others are available in scikit-learn.

## Comparing numbers

Before discussing the details of scaling with respect to machine learning, let's begin with a practical example comparing numbers with different scales. The number 1 million is larger than 10, but is an object weighing 1 million milligrams heavier than an object that weights 10 kilograms? The scale obviously matters when comparing numbers and can make a huge difference when interpreting values. One milligram is $10^{-6}$ kilograms and therefore 1 million milligrams reduces to just 1 kilogram, thus an object weighing 10 kilograms is heavier than one weighing 1 million milligrams.

### Distance calculations in machine learning

One of the clearest examples of where scale matters is during the calculation of distance between two observations. We saw this with the k-nearest neighbors model when using the overall quality and above ground living area as the two features in the model. Overall quality ranges from 1 to 10 while ground living area is typically in the thousands. The difference in overall quality between houses is inconsequential compared to the difference in above ground living area.

In order to make comparisons between features more accurate, we transform them in such a manner that they are all approximately on the same scale. This is what is referred to as **feature scaling**.

## Common feature scaling

The two basic kinds of feature scaling that are common for continuous features are standardization and min-max. Each one scales the features in a different manner.

### Standardization

Standardization transforms each feature such that the mean is 0 and standard deviation is 1. Many of the machine learning estimators require that the features are standardized in order for the algorithms that learn from the data to work appropriately.

The formula for standardization is simple. First, the mean is subtracted from each observation. This value is then divided by the standard deviation. If the feature is at all similar to a normal distribution, then the values will typically be between -3 and 3.

### Min-Max scaling

In min-max scaling, each feature is transformed such that each value falls between a given minimum and maximum value. By default, this minimum and maximum are 0 and 1, but they can be any number.

## Feature scaling in scikit-learn

Let's begin feature scaling in scikit-learn by reading in our housing data, selecting four columns, each with a different range of values.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
housing = pd.read_csv('../data/housing_sample.csv')
cols = ['GrLivArea', 'GarageArea', 'LotFrontage', 'OverallQual']
X = housing[cols]
y = housing['SalePrice']
X.head()

It's clear from just the first few rows that each feature has a different range of values. We confirm this by finding the first and third quartiles along with the median of each distribution with the `quantile` method.

In [ ]:
X.quantile([.25, .5, .75]).astype('int')

It should now be clear that each feature has a different distribution of values. Before we transform each feature to the same scale, we need to fill any missing values. Let's do this with the `SimpleImputer`. Note, that we overwrite the variable name `X` so that it now references the imputed array returned from the `fit_transform` method. Its first three rows are outputted below.

In [ ]:
from sklearn.impute import SimpleImputer
si = SimpleImputer(strategy='mean')
X = si.fit_transform(X)
X[:3]

### The `StandardScaler` transformer

scikit-learn has several transformers available to do feature scaling and are all found in the preprocessing module. Let's import `StandardScaler` and use the three-step process to learn from the features.

In [ ]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
ss.fit(X)

The `StandardScaler` estimator is relatively simple and by default learns the mean and variance for each feature. Let's discover what it learned by accessing the `mean_` and `var_` attributes.

In [ ]:
ss.mean_.round(1)

The variance is what is reported, but this is just the square of the standard deviation.

In [ ]:
ss.var_

Remember, no transformation happens during a call to the `fit` method. All estimators learn something from the data and the `StandardScaler` is no different. It learned the mean and variance when `fit` was called. To actually transform the data, you must call the `transform` method. We assign this result to the variable name `X_scaled` and output the first few rows.

In [ ]:
X_scaled = ss.transform(X)
X_scaled[:3].round(3)

As with all transformers, the learning from data (`fit`) and transforming (`transform`) can be combined into a single step with the `fit_transform` method.

In [ ]:
X_scaled = ss.fit_transform(X)
X_scaled[:3]

Let's verify that each feature of the transformed data has a mean of 0 and a standard deviation of 1.

In [ ]:
X_scaled.mean(axis=0)

In [ ]:
X_scaled.std(axis=0)

Box plots can help get a sense of the distributions for each feature. Below, we make a box plot of the original and scaled data.

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10))
ax1.boxplot(X, labels=cols)
ax1.set_title('Original Data')
ax2.boxplot(X_scaled, labels=cols)
ax2.set_title('Scaled Data');

### The `MinMaxScaler` transformer

The `MinMaxScaler` works similarly as `StandardScaler`. By default, it transforms each feature so that all values are between 0 and 1. Let's run the three step-process using `fit_transform` as the final step.

In [ ]:
from sklearn.preprocessing import MinMaxScaler
mms = MinMaxScaler()
X_scaled2 = mms.fit_transform(X)
X_scaled2[:3]

Let's use a box plot to see the new distribution and verify that the values are between 0 and 1.

In [ ]:
fig, ax = plt.subplots(figsize=(14, 5))
ax.boxplot(X_scaled2, labels=cols);

You can specify a different range for the features by passing a tuple as the first argument to the constructor. We specify the range to be between -2 and 2.

In [ ]:
mms = MinMaxScaler((-2, 2))
X_scaled3 = mms.fit_transform(X)

Let's verify that the minimum and maximum

In [ ]:
X_scaled3.min(axis=0)

In [ ]:
X_scaled3.max(axis=0)

### Not as robust to outliers

One issue with scaling based on the minimum and maximum values is that a single outlier can compress most of the information into just a fraction of the range. For example, if one house had a value of 50,000 for above ground living area (about ten times the current maximum), then there would be very little spread in the data when scaled. All of the house would be between 0 and .1 with a single value at 1.

### Use standardization except with neural networks

Standardizing the input data is generally a better strategy that min-max scaling as it handles outliers better, but in practice, the implementation of many neural networks work better when using min-max scaling.

## Machine learning with scaled features

Let's take our scaled features and use them as our new input into the same machine learning methods that we've seen before. To determine if there is a difference, we will provide the cross validated score of each model trained on both the scaled and original data.

### Linear regression

Let's determine whether the scaled features improve the model fit with linear regression. We continue to set the random state of the `KFold` splitter to 123 so that each time we execute `cross_val_score`, the same datasets are used for training and testing.

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold, cross_val_score
lr = LinearRegression()
kf = KFold(n_splits=5, shuffle=True, random_state=123)

Let's call `cross_val_score` on the original data with just the missing values filled.

In [ ]:
cross_val_score(lr, X, y, cv=kf).mean()

Let's compare this $R^2$ score to a model fit with the scaled data.

In [ ]:
cross_val_score(lr, X_scaled, y, cv=kf).mean()

The scores are the exact same showing that the scale of the features does not impact this kind of linear regression. However, we will soon examine penalized linear regression where a penalty is imposed on the model that is proportional to the size of the coefficients. In this form, the scaling makes a difference to how much learning takes place and will impact $R^2$.

### Decision Trees

Let's complete the same procedure with a decision tree that is built with a maximum depth of 5. First, we calculate $R^2$ on the original data.

In [ ]:
from sklearn.tree import DecisionTreeRegressor
dtr = DecisionTreeRegressor(max_depth=5, random_state=999)
cross_val_score(dtr, X, y, cv=kf).mean()

Notice, that the `random_state` of the `DecisionTreeRegressor` is set. Whenever there are multiple decisions that are tied for the best split, one of the decisions is selected at random. To make this decision the same, set the `random_state` parameter to an integer. Now, let's calculate our score on the original data.

In [ ]:
cross_val_score(dtr, X_scaled, y, cv=kf).mean()

Again, these scores are equal (or very nearly equal). Scaling the features has no impact on where the best splits occur. Unfortunately, there is a bug in scikit-learn and decisions made right on the boundary may not evaluate properly due to floating point inexactness so you might have scaled data that does not return the same exact score, which is the case here.

### K-Nearest Neighbors

As we saw in the chapter on k-nearest neighbors, the different sizes of the features impacts the distance calculation and changes which observations are deemed as the nearest neighbors. Let's show that the scores and neighbors are different with scaled data. We begin by finding the cross-validated $R^2$ for the original data.

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
knr = KNeighborsRegressor(n_neighbors=5)
cross_val_score(knr, X, y, cv=kf).mean()

Let's compare this $R^2$ score to a model fit with the scaled data.

In [ ]:
cross_val_score(knr, X_scaled, y, cv=kf).mean()

In this instance, there is a huge difference in the scores and it is due directly to the change in calculated distance. The feature `OverallQual` is a strong predictor of sale price. With the original data, it had almost no influence on the distance calculation as ground living area and garage area were around 2-3 orders of magnitude larger.

#### Finding the nearest neighbors

The `KNeighborsRegressor` estimator has the ability to return the nearest neighbor of each point predicted. We must first call the `fit` method on the original data so that we have a trained model. Remember, `cross_val_score` does not return a fitted model.

In [ ]:
knr.fit(X, y)

There exists a `kneighbors` method that finds the k-nearest neighbors (5 in this case) from a given observation. We pass it the first observation as a two-dimensional array, `X[[0]]`. It returns the integer index of the five nearest neighbors from this first observation as well as the distance itself. We assign each of the returned values to a separate variable.

In [ ]:
X[[0]]

In [ ]:
dist_orig, idx_orig = knr.kneighbors(X[[0]])

Let's take a look at the location of each of the five nearest neighbors.

In [ ]:
idx_orig

The nearest neighbor to the first observation is itself, as indicated by the index 0 in the above array. The nearest neighbor will always be itself if it was used in the training data as it was here. The next nearest neighbor is at index 1,207. Let's take a look at the feature values for each of these five nearest neighbors. Since the `idx_orig` was given as a two-dimensional array, we select it down to a single dimension in order to make a proper subset selection in the cell that follows.

In [ ]:
idx_orig = idx_orig[0]
idx_orig

Now that we have a one-dimensional array, we can select the feature values of the five nearest neighbors properly.

In [ ]:
X[idx_orig]

As you would expect, all of the values for each feature are close to the first observation. Let's calculate the raw distance between the first observation and its nearest neighbors.

In [ ]:
raw_dist_orig = X[idx_orig] - X[0]
raw_dist_orig

We can now compute the Euclidean distance from each neighbor to the first observation.

In [ ]:
np.sqrt((raw_dist_orig ** 2).sum(axis=1))

This is the same distance reported by scikit-learn that we assigned to the `dist_orig` variable name above.

In [ ]:
dist_orig

Let's fit a new model with the scaled data and return the indexes and distances for the five nearest neighbors.

In [ ]:
knr_scaled = KNeighborsRegressor(n_neighbors=5)
knr_scaled.fit(X_scaled, y)
dist_scaled, idx_scaled = knr_scaled.kneighbors(X_scaled[[0]])
idx_scaled = idx_scaled[0]
idx_scaled

All of the nearest neighbors (other than itself) are different from those that were found using the original data. Let's take a look at the original feature values of the nearest neighbors based on distance calculated from the scaled data.

In [ ]:
X[idx_scaled]

Notice that the overall quality values are all the same. Having a difference of even one with overall quality is relatively large because the raw numbers are so low. Let's also look at the actual scaled values. They are particularly hard to decipher.

In [ ]:
X_scaled[idx_scaled].round(2)

###  z-scores - different terminology

A different term for standardization, but one that carries an identical meaning, is **z-score**. The above array shows the z-scores of each observation. Although these scores are hard to decipher on their own, they do give us a relative metric - how many standard deviations away from the mean each feature value is. Let's take a look at the first three rows of z-scores.

In [ ]:
X_scaled[:3].round(2)

The first observation has a value of .37 for the first column. This house has an above ground living area that is .37 standard deviations above the mean. This informs us that this house is larger than average. It's lot frontage is -.22 or slightly below average.

The second house has above ground living area that is .48 standard deviations below the mean, but has a lot frontage that is larger than average.

The benefit of z-scores is that you can immediately tell whether each value for each feature is above or below average and to what degree. This wouldn't be easily possible by just looking at the raw data.

### Inverting the scaled data

Many transformers are equipped with an `inverse_transform` method that takes in the transformed data and returns the original. Here, we return our scaled data back to their original scales.

In [ ]:
ss.inverse_transform(X_scaled)

### Scaling effects with other machine learning models

Thus far, we've seen that scaling had effects on k-nearest neighbors but not linear regression or decision trees. Below are a list of other common supervised regression models and a comment on whether or not scaling the input data has any effect.

* Penalized Regression - Major effect - the penalty is proportional to the size of the coefficient
* Random Forests - No effect - random forests are just a collection of decision trees 
* Support Vector Machines - Major effect - support vector machines calculate distance between points and a separating hyper-plane.
* Neural networks - Major effect - nearly all neural networks perform better

## Linear regression coefficients using scaled data

Even though the predictions and $R^2$ value will not change when scaling the data with linear regression, the coefficients and their interpretation will change. Let's fit a linear regression model to our original dataset and access the coefficients.

In [ ]:
lr = LinearRegression()
lr.fit(X, y)
lr.coef_.round()

Building a model using the scaled data results in the following coefficients.

In [ ]:
lr_scaled = LinearRegression()
lr_scaled.fit(X_scaled, y)
lr_scaled.coef_.round(-2)

The ground living area had the lowest weight for the model using the original data, but the second highest when using the scaled data giving it some evidence that it is an important feature. The weight for overall quality remained high, but is no longer two orders of magnitude larger than the other coefficients.

### Model interpretation has changed

The coefficients of the model now have a different interpretation. Before, they corresponded to a one unit increase in the underlying feature. For instance, for every one foot of extra lot frontage (3rd feature) a corresponding increase in price of 159 dollars would be expected (assuming all other features stayed the same). 

Now, this one unit-increase is a one-standard deviation increase. Let's revisit the standard deviation of each feature, which was stored in the `var_` attribute of our `StandardScaler` object.

In [ ]:
np.sqrt(ss.var_)

Knowing this standard deviation, we can say from our second model, that a 525 square foot (1 standard deviation) increase in ground living area corresponds to a 25,000 dollar increase in the sale price. We can even show that dividing each coefficient by the standard deviation results in the same coefficients found in the original data.

In [ ]:
lr_scaled.coef_ / np.sqrt(ss.var_)

In [ ]:
lr.coef_

## Exercises

### Exercise 1

<span  style="color:green; font-size:16px">What are the cross-validated results when using min-max scaling for k-nearest neighbors? Does it perform better/worse than standardization?</span>

### Exercise 2

<span  style="color:green; font-size:16px">Take a look at the `RobustScaler` which scales by subtracting the mean and dividing by the interquartile range (difference between the 75th and 25th percentiles). What are the cross-validated scores on linear regression and knearest-neighbors. Does the `RobustScaler` change the value of $R^2$</span>

### Exercise 3

<span  style="color:green; font-size:16px">Take a look at the `PowerTransformer` applies a transformation to each column so that it appears more like a normal distribution with mean 0 and standard deviation of 1. What are the cross-validated results when using this transformer</span>

### Exercise 4

<span  style="color:green; font-size:16px">Assign the transformed input data for each of `StandardScaler`, `RobustScaler` and `PowerTransformer` to three different variable names. Create boxplots of each with pandas by uncommenting the following line and replacing the variable `scaled_data` with your variable name.</span>

In [ ]:
# pd.DataFrame(scaled_data).plot(kind='box')